In [9]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import requests
from zipfile import ZipFile
from io import BytesIO

In [12]:
def download_dataset():
    # URLs for the CSV files
    books_url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-Books.csv'
    ratings_url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-Book-Ratings.csv'

    # Download books CSV
    response_books = requests.get(books_url)
    books = pd.read_csv(BytesIO(response_books.content), encoding='ISO-8859-1', sep=';', error_bad_lines=False, warn_bad_lines=False)

    # Download ratings CSV
    response_ratings = requests.get(ratings_url)
    ratings = pd.read_csv(BytesIO(response_ratings.content), encoding='ISO-8859-1', sep=';', error_bad_lines=False, warn_bad_lines=False)

    return books, ratings


In [13]:
books, ratings = download_dataset()

books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

ratings.columns = ['userID', 'ISBN', 'bookRating']


merged_df = pd.merge(ratings, books, on='ISBN')

ratings_per_book = merged_df['bookTitle'].value_counts()
popular_books = ratings_per_book[ratings_per_book >= 100].index
merged_df = merged_df[merged_df['bookTitle'].isin(popular_books)]

pivot_df = merged_df.pivot_table(index='bookTitle', columns='userID', values='bookRating').fillna(0)

book_matrix = csr_matrix(pivot_df.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_matrix)


<ipython-input-12-15f318d6a8d0>:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(BytesIO(response_books.content), encoding='ISO-8859-1', sep=';', error_bad_lines=False, warn_bad_lines=False)
<ipython-input-12-15f318d6a8d0>:8: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(BytesIO(response_books.content), encoding='ISO-8859-1', sep=';', error_bad_lines=False, warn_bad_lines=False)
<ipython-input-12-15f318d6a8d0>:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings = pd.read_csv(BytesIO(response_ratings.content), encoding='ISO-8859-1', sep=';', error_bad_lines=False, warn_bad_lines=False)
<ipython-input-12-15f318d6a8d0>:12: FutureWarning: The warn_bad_lines argument

ValueError: Length mismatch: Expected axis has 1 elements, new values have 8 elements

In [14]:
def get_recommends(book = ""):
    # Find the index of the book
    book_list = list(pivot_df.index)
    book_idx = book_list.index(book)

    # Get distances and indices of the nearest neighbors
    distances, indices = model_knn.kneighbors(book_matrix[book_idx], n_neighbors = 6)

    # Create a list of recommended books and their distances
    recommends = [book]
    for i in range(1, len(distances.flatten())):
        recommends.append([book_list[indices.flatten()[i]], distances.flatten()[i]])

    return recommends